### PCA with Tensorflow

In this notebook, we calculate a simple PCA from 2d $\rightarrow$ 1d by minimizing the reconstruction error.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
X = np.reshape(np.asarray((-1.64,-0.4,-1.84,-0.9,-0.84,0.1,2.16,0.1,2.16,1.1)),(5,2))
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
res = pca.fit_transform(X)
W = pca.transform(np.identity(2))
W #The transformation

array([[ 0.95445112,  0.29836731],
       [ 0.29836731, -0.95445112]])

### PCA via reconstruction error

The PCA can be seen as an autoecoder. Here we go from 2D to 1D via the transformation $W_1$. $W_1$ is obtained from the full transformation matrix by keeping the first column corresponding to the largest eigenvector.    

$X \overbrace{\longrightarrow}^{W_1} X_{red} \overbrace{\longrightarrow}^{W_1'}  X_{rec}$

##### Approximate Reconstruction
The following numpy code calculates the reconstruction error.

In [3]:
W1 = np.reshape(W[:,0],(2,1))
#np.matmul(np.matmul(X, W1),np.transpose(W1))
X_rec = np.matmul(np.matmul(X, W1), np.transpose(W1))
np.sum((X - X_rec)**2)

0.693238275834537

### Implement a PCA using TensorFlow

Calculate the PCA in tensorflow, by reducing the reconstruction error as shown above using TensorFlow.

In [6]:
tf.reset_default_graph()
X_ = tf.placeholder(name='X', shape=(None, 2), dtype='float32')
W = tf.get_variable('W', shape=(2,1))
X_rec_ = tf.matmul(tf.matmul(X_, W), tf.transpose(W))
loss_ = tf.reduce_sum((X_rec_ - X_)**2)

In [8]:
train_op = tf.train.GradientDescentOptimizer(1e-3).minimize(loss_)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(200):
        _, d, W_tf = sess.run([train_op, loss_, W], feed_dict={X_: X})
        if (i%10 == 0):
            print(d)
            
print("The transformation is {}".format(W_tf))            

12.40214
5.871645
3.7112417
2.3719084
1.5846151
1.1539247
0.9279841
0.8119956
0.75309914
0.72335654
0.7083779
0.700845
0.6970592
0.6951574
0.69420207
0.6937223
0.6934813
0.6933603
0.69329953
0.693269
The transformation is [[-0.95416546]
 [-0.2992795 ]]
